In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os
import numpy as np
from sklearn.utils import class_weight
import matplotlib.pyplot as plt

print("✅ TensorFlow version:", tf.__version__)

CLASSES = ['Eczema', 'melanoma', 'Normal_Skin', 'Pigmentation', 'Rosacea', 'Scar', 'Vitiligo']
DATASET_PATH = "Dataset_Minor_Skin_Resized"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32


✅ TensorFlow version: 2.20.0


In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='training',
    class_mode='categorical'
)

val_gen = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='validation',
    class_mode='categorical'
)

print("📁 Class mapping:", train_gen.class_indices)


Found 3520 images belonging to 7 classes.
Found 876 images belonging to 7 classes.
📁 Class mapping: {'Eczema': 0, 'Normal_Skin': 1, 'Pigmentation': 2, 'Rosacea': 3, 'Scar': 4, 'Vitiligo': 5, 'melanoma': 6}


In [8]:
# --- Compute Class Weights for Imbalanced Dataset ---
from sklearn.utils import class_weight

class_weights = dict(enumerate(class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)))

print("📊 Computed class weights:", class_weights)


📊 Computed class weights: {0: np.float64(1.5714285714285714), 1: np.float64(1.6065723413966226), 2: np.float64(0.6869633099141296), 3: np.float64(1.5714285714285714), 4: np.float64(0.7857142857142857), 5: np.float64(1.6541353383458646), 6: np.float64(0.564373897707231)}


In [9]:
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(train_gen.class_indices), activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,422,855 (9.24 MB)

 Trainable params: 164,871 (644.03 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [10]:
# Define callbacks
checkpoint = ModelCheckpoint(
    "best_model_v2.h5",
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

# --- Train the Model ---
EPOCHS = 25

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stop, reduce_lr],
    class_weight=class_weights  # ✅ Balances training
)


C:\Users\SHAAN (0901AI231061)\AppData\Roaming\Python\Python313\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step - accuracy: 0.3027 - loss: 1.8929 
Epoch 1: val_accuracy improved from None to 0.68836, saving model to best_model_v2.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 37s 321ms/step - accuracy: 0.4406 - loss: 1.5602 - val_accuracy: 0.6884 - val_loss: 0.9791 - learning_rate: 1.0000e-04
Epoch 2/25
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.6662 - loss: 0.9795 
Epoch 2: val_accuracy improved from 0.68836 to 0.74315, saving model to best_model_v2.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 37s 336ms/step - accuracy: 0.6878 - loss: 0.9283 - val_accuracy: 0.7432 - val_loss: 0.7760 - learning_rate: 1.0000e-04
Epoch 3/25
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step - accuracy: 0.7614 - loss: 0.7376 
Epoch 3: val_accuracy improved from 0.74315 to 0.80708, saving model to best_model_v2.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 37s 337ms/step - accuracy: 0.7747 - loss: 0.7091 - val_accuracy: 0.8071 - val_loss: 0.6231 - learning_rate: 1.0000e-04
Epoch 4/25
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.8056 - loss: 0.6246 
Epoch 4: val_accuracy improved from 0.80708 to 0.82078, saving model to best_model_v2.h5


...

110/110 ━━━━━━━━━━━━━━━━━━━━ 37s 332ms/step - accuracy: 0.8832 - loss: 0.3525 - val_accuracy: 0.8619 - val_loss: 0.4110 - learning_rate: 1.0000e-04
Epoch 14/25
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.8985 - loss: 0.3185 
Epoch 14: val_accuracy improved from 0.86187 to 0.86416, saving model to best_model_v2.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 36s 329ms/step - accuracy: 0.9026 - loss: 0.3114 - val_accuracy: 0.8642 - val_loss: 0.3928 - learning_rate: 1.0000e-04
Epoch 15/25
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step - accuracy: 0.9055 - loss: 0.3088 
Epoch 15: val_accuracy did not improve from 0.86416
110/110 ━━━━━━━━━━━━━━━━━━━━ 37s 338ms/step - accuracy: 0.9037 - loss: 0.3169 - val_accuracy: 0.8584 - val_loss: 0.4269 - learning_rate: 1.0000e-04
Epoch 16/25
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.8958 - loss: 0.3232 
Epoch 16: val_accuracy improved from 0.86416 to 0.86530, saving model to best_model_v2.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 37s 333ms/step - accuracy: 0.8989 - loss: 0.3136 - val_accuracy: 0.8653 - val_loss: 0.4061 - learning_rate: 1.0000e-04
Epoch 17/25
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.9151 - loss: 0.3056 
Epoch 17: val_accuracy improved from 0.86530 to 0.86872, saving model to best_model_v2.h5



Epoch 17: ReduceLROnPlateau reducing learning rate to 2.9999999242136255e-05.
110/110 ━━━━━━━━━━━━━━━━━━━━ 36s 328ms/step - accuracy: 0.9128 - loss: 0.2938 - val_accuracy: 0.8687 - val_loss: 0.4064 - learning_rate: 1.0000e-04
Epoch 18/25
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.9153 - loss: 0.2547 
Epoch 18: val_accuracy improved from 0.86872 to 0.87329, saving model to best_model_v2.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 36s 328ms/step - accuracy: 0.9125 - loss: 0.2711 - val_accuracy: 0.8733 - val_loss: 0.4088 - learning_rate: 3.0000e-05
Epoch 19/25
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.9174 - loss: 0.2918 
Epoch 19: val_accuracy did not improve from 0.87329
110/110 ━━━━━━━━━━━━━━━━━━━━ 36s 329ms/step - accuracy: 0.9142 - loss: 0.2799 - val_accuracy: 0.8733 - val_loss: 0.4010 - learning_rate: 3.0000e-05
Epoch 19: early stopping
Restoring model weights from the end of the best epoch: 14.


In [11]:
# --- Phase 2: Fine-Tuning (Final Optimization) ---

# Unfreeze last 30 layers of MobileNetV2 for domain-specific adaptation
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Recompile with smaller learning rate to avoid weight destruction
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training with same callbacks & class balancing
fine_tune_epochs = 10
total_epochs = len(history.epoch) + fine_tune_epochs

history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=total_epochs,
    initial_epoch=len(history.epoch),
    callbacks=[checkpoint, early_stop, reduce_lr],
    class_weight=class_weights
)


Epoch 20/29
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step - accuracy: 0.4831 - loss: 1.4186 
Epoch 20: val_accuracy did not improve from 0.87329
110/110 ━━━━━━━━━━━━━━━━━━━━ 43s 359ms/step - accuracy: 0.6017 - loss: 1.0576 - val_accuracy: 0.8619 - val_loss: 0.4682 - learning_rate: 1.0000e-05
Epoch 21/29
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step - accuracy: 0.8015 - loss: 0.5729 
Epoch 21: val_accuracy did not improve from 0.87329
110/110 ━━━━━━━━━━━━━━━━━━━━ 40s 359ms/step - accuracy: 0.8241 - loss: 0.5385 - val_accuracy: 0.8505 - val_loss: 0.4895 - learning_rate: 1.0000e-05
Epoch 22/29
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step - accuracy: 0.8599 - loss: 0.4685 
Epoch 22: val_accuracy did not improve from 0.87329

Epoch 22: ReduceLROnPlateau reducing learning rate to 2.9999999242136253e-06.
110/110 ━━━━━━━━━━━━━━━━━━━━ 40s 366ms/step - accuracy: 0.8653 - loss: 0.4426 - val_accuracy: 0.8482 - val_loss: 0.4875 - learning_rate: 1.0000e-05
Epoch 23/29
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/s

In [12]:
model.save("skin_disease_model_mobilenetv2_final.keras")
print("✅ Model saved successfully!")


✅ Model saved successfully!


In [13]:
loaded_model = tf.keras.models.load_model("skin_disease_model_mobilenetv2_final.keras")
print("✅ Model reloaded successfully!")


✅ Model reloaded successfully!


In [14]:
val_loss, val_acc = loaded_model.evaluate(val_gen)
print(f"✅ Validation Accuracy: {val_acc*100:.2f}%")
print(f"✅ Validation Loss: {val_loss:.4f}")


28/28 ━━━━━━━━━━━━━━━━━━━━ 8s 243ms/step - accuracy: 0.8493 - loss: 0.4892
✅ Validation Accuracy: 84.93%
✅ Validation Loss: 0.4892


In [16]:
print("Train class indices:", train_gen.class_indices)
print("Val class indices:", val_gen.class_indices)



Train class indices: {'Eczema': 0, 'Normal_Skin': 1, 'Pigmentation': 2, 'Rosacea': 3, 'Scar': 4, 'Vitiligo': 5, 'melanoma': 6}
Val class indices: {'Eczema': 0, 'Normal_Skin': 1, 'Pigmentation': 2, 'Rosacea': 3, 'Scar': 4, 'Vitiligo': 5, 'melanoma': 6}


In [17]:
val_loss, val_acc = model.evaluate(val_gen)
print(f"✅ Corrected Validation Accuracy: {val_acc*100:.2f}%")


28/28 ━━━━━━━━━━━━━━━━━━━━ 7s 242ms/step - accuracy: 0.8459 - loss: 0.4591
✅ Corrected Validation Accuracy: 84.59%


In [18]:
# ✅ Ensure validation data order is not shuffled
val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    "Dataset_Minor_Skin_Resized",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # 🚨 Important fix
)

# ✅ Evaluate again
val_loss, val_acc = model.evaluate(val_gen)
print(f"✅ Corrected Validation Accuracy: {val_acc*100:.2f}%")

# ✅ True classification report (aligned)
import numpy as np
from sklearn.metrics import classification_report

y_true = val_gen.classes
y_pred = np.argmax(model.predict(val_gen), axis=1)

target_names = list(val_gen.class_indices.keys())
print("\n📊 Correct Classification Report:")
print(classification_report(y_true, y_pred, target_names=target_names))


Found 4396 images belonging to 7 classes.
  1/138 ━━━━━━━━━━━━━━━━━━━━ 22s 166ms/step - accuracy: 0.7812 - loss: 0.6621

C:\Users\SHAAN (0901AI231061)\AppData\Roaming\Python\Python313\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


138/138 ━━━━━━━━━━━━━━━━━━━━ 18s 133ms/step - accuracy: 0.9170 - loss: 0.2472
✅ Corrected Validation Accuracy: 91.70%
138/138 ━━━━━━━━━━━━━━━━━━━━ 20s 143ms/step

📊 Correct Classification Report:
              precision    recall  f1-score   support

      Eczema       0.70      0.87      0.78       399
 Normal_Skin       0.97      0.84      0.90       391
Pigmentation       0.94      0.93      0.94       914
     Rosacea       0.83      0.88      0.85       399
        Scar       0.93      0.92      0.92       800
    Vitiligo       0.98      0.83      0.90       380
    melanoma       0.98      0.99      0.98      1113

    accuracy                           0.92      4396
   macro avg       0.91      0.89      0.90      4396
weighted avg       0.92      0.92      0.92      4396

